In [1]:
import cv2
import numpy as np
import math
import time
from scipy.ndimage.filters import gaussian_filter
import matplotlib.pyplot as plt
import matplotlib
import torch
from torchvision import transforms

import util
from model import BodyposeModel

from torchsummary import summary

/tmp/ipykernel_8735/1643972621.py:5: DeprecationWarning: Please use `gaussian_filter` from the `scipy.ndimage` namespace, the `scipy.ndimage.filters` namespace is deprecated.
  from scipy.ndimage.filters import gaussian_filter


In [2]:
class Body(object):
    def __init__(self, model_path: str):
        self.model = BodyposeModel()
        if torch.cuda.is_available():
            self.model = self.model.cuda()
        model_dict = util.transfer(self.model, torch.load(model_path))
        self.model.load_state_dict(model_dict)
        self.model.eval()

In [3]:
body = Body('../model/body_pose_model.pth')

In [6]:
summary(body.model, (3, 640, 480))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 640, 480]           1,792
              ReLU-2         [-1, 64, 640, 480]               0
            Conv2d-3         [-1, 64, 640, 480]          36,928
              ReLU-4         [-1, 64, 640, 480]               0
         MaxPool2d-5         [-1, 64, 320, 240]               0
            Conv2d-6        [-1, 128, 320, 240]          73,856
              ReLU-7        [-1, 128, 320, 240]               0
            Conv2d-8        [-1, 128, 320, 240]         147,584
              ReLU-9        [-1, 128, 320, 240]               0
        MaxPool2d-10        [-1, 128, 160, 120]               0
           Conv2d-11        [-1, 256, 160, 120]         295,168
             ReLU-12        [-1, 256, 160, 120]               0
           Conv2d-13        [-1, 256, 160, 120]         590,080
             ReLU-14        [-1, 256, 1